# Multi-variate time series classification with sktime-dl using InceptionTime
[Github](https://github.com/sktime/sktime-dl)

In this notebook, we use sktime-dl to perform for multi-variate time series classification by deep learning.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sktime.datasets import load_basic_motions
from sktime.classification.deep_learning.inceptiontime import InceptionTimeClassifier

sns.set_style('whitegrid') 

/home/weiss/.conda/envs/python39/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


# Load a dataset
The Basic Motions dataset, from [timeseriesclassification.com](http://www.timeseriesclassification.com), has time series in six dimensions.

In [2]:
X_train, y_train = load_basic_motions(split='train', return_X_y=True)
X_test, y_test = load_basic_motions(split='test', return_X_y=True)
print(X_train.shape)
print(X_test.shape)
X_train.head()

(40, 6)
(40, 6)


,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5
0,0 0.079106 1 0.079106 2 -0.903497 3...,0 0.394032 1 0.394032 2 -3.666397 3...,0 0.551444 1 0.551444 2 -0.282844 3...,0 0.351565 1 0.351565 2 -0.095881 3...,0 0.023970 1 0.023970 2 -0.319605 3...,0 0.633883 1 0.633883 2 0.972131 3...
1,0 0.377751 1 0.377751 2 2.952965 3...,0 -0.610850 1 -0.610850 2 0.970717 3...,0 -0.147376 1 -0.147376 2 -5.962515 3...,0 -0.103872 1 -0.103872 2 -7.593275 3...,0 -0.109198 1 -0.109198 2 -0.697804 3...,0 -0.037287 1 -0.037287 2 -2.865789 3...
2,0 -0.813905 1 -0.813905 2 -0.424628 3...,0 0.825666 1 0.825666 2 -1.305033 3...,0 0.032712 1 0.032712 2 0.826170 3...,0 0.021307 1 0.021307 2 -0.372872 3...,0 0.122515 1 0.122515 2 -0.045277 3...,0 0.775041 1 0.775041 2 0.383526 3...
3,0 0.289855 1 0.289855 2 -0.669185 3...,0 0.284130 1 0.284130 2 -0.210466 3...,0 0.213680 1 0.213680 2 0.252267 3...,0 -0.314278 1 -0.314278 2 0.018644 3...,0 0.074574 1 0.074574 2 0.007990 3...,0 -0.079901 1 -0.079901 2 0.237040 3...
4,0 -0.123238 1 -0.123238 2 -0.249547 3...,0 0.379341 1 0.379341 2 0.541501 3...,0 -0.286006 1 -0.286006 2 0.208420 3...,0 -0.098545 1 -0.098545 2 -0.023970 3...,0 0.058594 1 0.058594 2 0.175783 3...,0 -0.074574 1 -0.074574 2 0.114525 3...


In [3]:
# The class labels
np.unique(y_train)

array(['badminton', 'running', 'standing', 'walking'], dtype='<U9')

# Train a deep neural network classifier
Here we choose to use the InceptionTime classifier. Other classifiers provided by sktime-dl include MLP, ResNet, MCDCNN (Multi Channel Deep Convolutional Neural Network)

In [4]:
network = InceptionTimeClassifier(n_epochs=200, verbose=False)
network.fit(X_train, y_train)
network.score(X_test, y_test)

2022-07-14 11:22:31.226105: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-07-14 11:22:31.226168: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: tobi-legion
2022-07-14 11:22:31.226183: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: tobi-legion
2022-07-14 11:22:31.226385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.57.0
2022-07-14 11:22:31.226433: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.57.0
2022-07-14 11:22:31.226446: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.57.0
2022-07-14 11:22:31.226930: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-criti

1.0

# Grid Search
sktime-dl is compatible with scikit-learn and can use sklearn's GridSearchCV.

Here we search over two parameters, CNN kernel size and pooling size.

In [5]:
param_grid = {'nb_filters': [16, 32],
              'bottleneck_size': [16, 32]}
grid = GridSearchCV(network, param_grid=param_grid, cv=5)   
grid.fit(X_train, y_train)
    
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

2022-07-14 11:22:54.076996: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://c4c7fa62-cdfc-44a9-966d-2d20f17624c5/assets
INFO:tensorflow:Assets written to: ram://5801c504-6461-44b0-b798-c90f178835ce/assets
INFO:tensorflow:Assets written to: ram://67a565ac-00d1-43ea-bb76-410081e46e90/assets
INFO:tensorflow:Assets written to: ram://9920cf2e-30a9-4f28-b4db-5c9dfdd620ed/assets
INFO:tensorflow:Assets written to: ram://09183403-54df-4b32-9562-854a6ce17a41/assets
INFO:tensorflow:Assets written to: ram://6df1237e-51d8-4a0b-8e26-1bdefd7014f3/assets
INFO:tensorflow:Assets written to: ram://9fafb7b5-f31e-4fbd-a810-29d703502981/assets
INFO:tensorflow:Assets written to: ram://7dbf3f10-b662-4b63-bada-ee242d749afc/assets
INFO:tensorflow:Assets written to: ram://76f77650-7b82-4534-aacd-ff95aed96243/assets
INFO:tensorflow:Assets written to: ram://543fba06-05f6-4ada-9fcf-685acd10b69f/assets
INFO:tensorflow:Assets written to: ram://0b9d5c52-d47a-44ca-a18e-8ad9d71b8f62/assets
INFO:tensorflow:Assets written to: ram://405ddcd4-ee46-4691-9456-